<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/california_housing/defiend_ShallowNN_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, termcolor, wrapt, astunparse, tensorboard, grpcio, scipy, opt-einsum, six, absl-py, protobuf, tensorflow-estimator, h5py, google-pasta, numpy, wheel, gast
Required-by: fancyimpute


In [2]:
%cd /content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
%ls

/content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
auto_keras_outputs/          defiend_ShallowNN_regressor.ipynb
autokeras_regressor.ipynb    fetch_data.ipynb
datasets/                    xgb_regressor.ipynb
defiend_DNN_regressor.ipynb


In [3]:
import cloudpickle, os, pprint

import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

## データ読み込み

In [4]:
F_COLS = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
TARGET_COL = 'MedHouseVal'

data_path = 'datasets/origin'
train_csv = 'train.csv'
val_csv = 'val.csv'
test_csv = 'test.csv'

train_df= pd.read_csv(os.path.join(data_path, train_csv), header=0)
val_df = pd.read_csv(os.path.join(data_path, val_csv), header=0)
test_df = pd.read_csv(os.path.join(data_path, test_csv), header=0)

print('train shape:{}'.format(train_df.shape))
print('validation shape:{}'.format(val_df.shape))
print('test shape:{}'.format(test_df.shape))

display(train_df.head())

train shape:(13209, 9)
validation shape:(3303, 9)
test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,4.4063,15.0,6.104430,1.003165,1237.0,3.914557,34.00,-117.50,1.285
1,3.7222,35.0,5.539326,1.030899,1152.0,3.235955,33.87,-117.96,1.875
2,1.1094,49.0,3.773585,1.144654,1448.0,4.553459,37.95,-121.28,0.525
3,3.3239,9.0,5.610955,1.092697,1691.0,2.375000,36.82,-119.85,0.913
4,5.9629,17.0,6.867133,1.097902,808.0,2.825175,38.58,-121.81,2.860


## Shallow Neural Networkモデルを構築

In [5]:
def model_fn(input_shape, hidden_size, output_size):
  '''モデルの定義'''
  model = Sequential([
    layers.Dense(hidden_size, activation='relu', input_shape=[input_shape]),
    layers.Dense(output_size)
  ])
  return model


model = model_fn(input_shape=len(F_COLS), hidden_size=64,
                 output_size=1)

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae', 'mse'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________


In [6]:
%time
BATCH_SIZE = 256
EPOCHS = 100

es = EarlyStopping(patience=3)

model.fit(x=train_df[F_COLS].values, y=train_df[TARGET_COL].values,
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          validation_data=(val_df[F_COLS].values, val_df[TARGET_COL].values),
          callbacks=[es]
          )

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs
Epoch 1/100
52/52 [==============================] - 0s 5ms/step - loss: 422638.3125 - mae: 443.7372 - mse: 422638.3125 - val_loss: 44012.1992 - val_mae: 141.4354 - val_mse: 44012.1992
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 12291.4287 - mae: 66.8137 - mse: 12291.4287 - val_loss: 2495.9775 - val_mae: 44.8956 - val_mse: 2495.9775
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 1054.8199 - mae: 19.5608 - mse: 1054.8199 - val_loss: 47.0933 - val_mae: 2.6940 - val_mse: 47.0933
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 55.5700 - mae: 2.9130 - mse: 55.5700 - val_loss: 23.2650 - val_mae: 2.8178 - val_mse: 23.2650
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 27.0311 - mae: 2.4494 - mse: 27.0311 - val_loss: 8.1673 - val_mae: 1.9677 - val_mse: 8.1673
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - l

## 前処理したデータの読み込み
行った前処理
- 標準化

In [7]:
std_data_path = 'datasets/std'
std_train_csv = 'std_train.csv'
std_val_csv = 'std_val.csv'
std_test_csv = 'std_test.csv'

std_train_df= pd.read_csv(os.path.join(std_data_path, std_train_csv), header=0)
std_val_df = pd.read_csv(os.path.join(std_data_path, std_val_csv), header=0)
std_test_df = pd.read_csv(os.path.join(std_data_path, std_test_csv), header=0)

print('std_train shape:{}'.format(std_train_df.shape))
print('std_validation shape:{}'.format(std_val_df.shape))
print('std_test shape:{}'.format(std_test_df.shape))

display(std_train_df.head())

std_train shape:(13209, 9)
std_validation shape:(3303, 9)
std_test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0.297052,-1.079924,0.251551,-0.185450,-0.170175,0.119535,-0.760080,1.026143,-0.667466
1,-0.062875,0.505159,0.040977,-0.132852,-0.242531,0.024947,-0.821013,0.796661,-0.152404
2,-1.437551,1.614717,-0.616988,0.082885,0.009438,0.208590,1.091352,-0.859600,-1.330936
3,-0.272433,-1.555449,0.067668,-0.015652,0.216291,-0.095059,0.561702,-0.146211,-0.992217
4,1.116028,-0.921416,0.535755,-0.005780,-0.535361,-0.032310,1.386643,-1.124003,0.707487


In [8]:
# 標準化器も読み込んでおく
def load_pkl(path):
  '''pklファイルの読み込み'''
  with open(path, 'rb') as f:
    obj = cloudpickle.load(f)
  return obj


pkl_name = 'std_scaler.pkl'
std_scaler = load_pkl(os.path.join(std_data_path, pkl_name))
std_scaler

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

## 前処理済みデータに対してモデルを構築

In [9]:
std_model = model_fn(input_shape=len(F_COLS), hidden_size=64,
                     output_size=1)

std_model.compile(loss='mse',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['mae', 'mse'])

std_model.fit(x=std_train_df[F_COLS].values, y=std_train_df[TARGET_COL].values,
              batch_size=BATCH_SIZE, epochs=EPOCHS,
              validation_data=(std_val_df[F_COLS].values, std_val_df[TARGET_COL].values),
              callbacks=[es]
              )

Epoch 1/100
52/52 [==============================] - 0s 4ms/step - loss: 0.6280 - mae: 0.6006 - mse: 0.6280 - val_loss: 0.5417 - val_mae: 0.5474 - val_mse: 0.5417
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 0.4496 - mae: 0.4984 - mse: 0.4496 - val_loss: 0.4720 - val_mae: 0.5013 - val_mse: 0.4720
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 0.3996 - mae: 0.4636 - mse: 0.3996 - val_loss: 0.4214 - val_mae: 0.4702 - val_mse: 0.4214
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 0.3633 - mae: 0.4393 - mse: 0.3633 - val_loss: 0.3837 - val_mae: 0.4423 - val_mse: 0.3837
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 0.3391 - mae: 0.4224 - mse: 0.3391 - val_loss: 0.3600 - val_mae: 0.4287 - val_mse: 0.3600
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 0.3232 - mae: 0.4109 - mse: 0.3232 - val_loss: 0.3439 - val_mae: 0.4155 - val_mse: 0.3439
Epoch 7/100
52/52 [===

## モデルの評価

### 評価指標の定義

In [10]:
SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]

def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result

### 推論結果の取得

In [11]:
def predict_multiple_times(model, inputs, input_names):
  '''何度か推論'''
  pred = {}
  for input, name in zip(inputs, input_names):
    pred[name] = model.predict(input)
  return pred

In [12]:
%time
reg_inputs = [train_df[F_COLS].values, val_df[F_COLS], test_df[F_COLS].values]
reg_input_names = [train_csv, val_csv, test_csv]

std_reg_inputs = [std_train_df[F_COLS].values, std_val_df[F_COLS].values, std_test_df[F_COLS].values]
std_input_names = [std_train_csv, std_val_csv, std_test_csv]


reg_preds = predict_multiple_times(model=model, inputs=reg_inputs, input_names=reg_input_names)
std_reg_preds = predict_multiple_times(model=std_model, inputs=std_reg_inputs, input_names=std_input_names)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


### 評価の実行

In [13]:
model_name = 'ShallowNN'
std_model_name = 'std_ShallowNN'

eval_scores = {model_name: {}, std_model_name: {}}

for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                          y_pred=reg_preds[csv_name], sklearn_metrics=SK_METRICS)

for csv_name, y_true in zip([std_train_csv, std_val_csv, std_test_csv], 
                            [std_train_df[TARGET_COL].values, std_val_df[TARGET_COL].values, std_test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=std_reg_preds[csv_name], sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': 0.4113507362237835,
                            'mean_squared_error': 0.8671976812652659,
                            'r2_score': 0.3552512656067024},
               'train.csv': {'explained_variance_score': 0.4382437418720341,
                             'mean_squared_error': 0.8216732977780793,
                             'r2_score': 0.3737983108131855},
               'val.csv': {'explained_variance_score': 0.42981381590755363,
                           'mean_squared_error': 0.8588002188503151,
                           'r2_score': 0.3813672975773672}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.7435970868416897,
                                    'mean_squared_error': 0.2629155355581765,
                                    'r2_score': 0.7435081450151091},
                   'std_train.csv': {'explained_variance_score': 0.7605143906602558,
                                     'mean_squared_error': 0.

## モデルの予測値を標準化から復元し評価する場合

In [14]:
def inverse_scale_values(values, scaler):
  '''scalerを使って値の復元'''
  return scaler.inverse_transform(values)


inverse_scale_preds = {}
# 目的変数をモデルの予測値に置き換え
for df, csv_name, key in zip([std_train_df, std_val_df, std_test_df], [std_train_csv, std_val_csv, std_test_csv], 
                             [train_csv, val_csv, test_csv]):
  df[TARGET_COL] = std_reg_preds[csv_name]
  tmp_df = pd.DataFrame(inverse_scale_values(df.values, std_scaler), columns=F_COLS+[TARGET_COL])
  inverse_scale_preds[key] = tmp_df[TARGET_COL].values

inverse_scale_preds

{'test.csv': array([2.05405413, 2.71168867, 3.0679269 , ..., 1.33230109, 3.23131995,
        4.16991808]),
 'train.csv': array([1.58030596, 1.70514985, 0.7200618 , ..., 1.27362621, 1.2391027 ,
        2.21615701]),
 'val.csv': array([1.06332521, 0.86363761, 1.29767183, ..., 0.9586947 , 1.20174339,
        0.85731437])}

In [15]:
# 標準化された値を復元した場合の評価スコア
for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=inverse_scale_preds[csv_name],
                                                              sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': 0.4113507362237835,
                            'mean_squared_error': 0.8671976812652659,
                            'r2_score': 0.3552512656067024},
               'train.csv': {'explained_variance_score': 0.4382437418720341,
                             'mean_squared_error': 0.8216732977780793,
                             'r2_score': 0.3737983108131855},
               'val.csv': {'explained_variance_score': 0.42981381590755363,
                           'mean_squared_error': 0.8588002188503151,
                           'r2_score': 0.3813672975773672}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.7435970868416897,
                                    'mean_squared_error': 0.2629155355581765,
                                    'r2_score': 0.7435081450151091},
                   'std_train.csv': {'explained_variance_score': 0.7605143906602558,
                                     'mean_squared_error': 0.